In [2]:
# Import libraries
import pymongo as md
from pymongo import MongoClient
from bson import ObjectId
import json
import pprint
import psycopg2
import pandas.io.sql as sqlio
import pandas as pd

In [2]:
createCovidTable = """
CREATE TABLE coviddata(
DataAsOf date,
StartDate date,
EndDate date,
Groups varchar,
Year integer,
Month integer,
HHSRegion integer,
State varchar,
PlaceOfDeath varchar,
AgeGroup varchar,
COVID19Deaths numeric,
TotalDeaths numeric,
PneumoniaDeaths numeric,
PneumoniaAndCOVID19Deaths numeric,
InfluenzaDeaths numeric,
PneumoniaInfluenzaOrCOVID19Deaths numeric
);
"""

createMentalTable = """
CREATE TABLE mentalstatedata(
Indicator VARCHAR,
Groups VARCHAR,
State VARCHAR,
Subgroup VARCHAR,
Phase VARCHAR,
TimePeriod numeric(5,2),
TimePeriodLabel VARCHAR,
TimePeriodStartDate date,
TimePeriodEndDate date,
Value numeric(5,2),
LowCI numeric(5,2),
HighCI numeric(5,2),
ConfidenceInterval numeric(5,2),
QuartileRange numeric(5,2)
);
"""

createNursingTable = """
CREATE TABLE nursingdata(
WeekEnding date,
ProviderName varchar,
ProviderCity varchar,
ResidentsTotalAdmissionsCOVID19 bigint,
ResidentsWeeklyConfirmedCOVID19 bigint,
ResidentsTotalConfirmedCOVID19 bigint,
ResidentsTotalSuspectedCOVID19 bigint,
ResidentsWeeklyAllDeaths bigint,
ResidentsTotalAllDeaths bigint,
ResidentsWeeklyCOVID19Deaths bigint,
ResidentsTotalCOVID19Deaths bigint,
NumberofAllBeds bigint,
TotalNumberofOccupiedBeds bigint,
StaffWeeklyConfirmedCOVID19 bigint,
StaffTotalConfirmedCOVID19 bigint,
StaffWeeklySuspectedCOVID19 bigint,
StaffTotalSuspectedCOVID19 bigint,
StaffTotalCOVID19Deaths varchar,
ShortageofNursingStaff varchar,
ShortageofClinicalStaff varchar,
ShortageofAides varchar,
ShortageofOtherStaff varchar,
AnyCurrentSupplyofN95Masks varchar,
OneWeekSupplyofN95Masks varchar,
AnyCurrentSupplyofSurgicalMasks varchar,
OneWeekSupplyofSurgicalMasks varchar,
AnyCurrentSupplyofEyeProtection varchar,
OneWeekSupplyofEyeProtection varchar,
OneWeekSupplyofGowns varchar,
AnyCurrentSupplyofHandSanitizer varchar,
TotalResidentConfirmedCOVID19CasesPer1000Residents float,
TotalResidentCOVID19DeathsPer1000Residents float,
TotalResidentsCOVID19DeathsasaPercentageofConfirmedCOVID19Cases float,
ThreeorMoreConfirmedCOVID19CasesThisWeekorInitialConfirmedCOVID19CasethisWeek varchar,
AbletoTestorObtainResourcestoTestAllCurrentResidentsWithinNext7Days varchar,
AbletoTestorObtainResourcestoTestAllStaffandorPersonnelWithinNext7Days varchar,
COVID19PointofCareTestsPerformedonResidentsSinceLastReport float,
COVID19PointofCareTestsPerformedonStaffandorPersonnelSinceLastReport float,
WeeklyResidentConfirmedCOVID19CasesPer1000Residents float

);
"""


createCovidHospitalTable="""
CREATE TABLE covidhospitaldata (
state    varchar    ,
date    date    ,
critical_staffing_shortage_today_yes    float    ,
critical_staffing_shortage_today_no    float    ,
critical_staffing_shortage_today_not_reported    float    ,
critical_staffing_shortage_anticipated_within_week_yes    float    ,
critical_staffing_shortage_anticipated_within_week_no    float    ,
critical_staffing_shortage_anticipated_within_week_not_reported    float    ,
hospital_onset_covid    float    ,
hospital_onset_covid_coverage    float    ,
inpatient_beds    float    ,
inpatient_beds_coverage    float    ,
inpatient_beds_used    float    ,
inpatient_beds_used_coverage    float    ,
inpatient_beds_used_covid    float    ,
inpatient_beds_used_covid_coverage    float    ,
previous_day_admission_adult_covid_confirmed_coverage    float    ,
previous_day_admission_adult_covid_suspected_coverage    float    ,
previous_day_admission_pediatric_covid_confirmed_coverage    float    ,
previous_day_admission_pediatric_covid_suspected_coverage    float    ,
staffed_adult_icu_bed_occupancy_coverage    float    ,
staffed_icu_adult_patients_confirmed_and_suspected_covid_coverage    float    ,
staffed_icu_adult_patients_confirmed_covid_coverage    float    ,
total_adult_patients_hospitalized_confirmed_and_suspected_covid_coverage    float    ,
total_adult_patients_hospitalized_confirmed_covid_coverage    float    ,
total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_coverage    float    ,
total_pediatric_patients_hospitalized_confirmed_covid_coverage    float    ,
total_staffed_adult_icu_beds_coverage    float    ,
inpatient_beds_utilization    float    ,
inpatient_beds_utilization_coverage    float    ,
inpatient_beds_utilization_numerator    float    ,
inpatient_beds_utilization_denominator    float    ,
percent_of_inpatients_with_covid    float    ,
percent_of_inpatients_with_covid_coverage    float    ,
percent_of_inpatients_with_covid_numerator    float    ,
percent_of_inpatients_with_covid_denominator    float    ,
inpatient_bed_covid_utilization    float    ,
inpatient_bed_covid_utilization_coverage    float    ,
inpatient_bed_covid_utilization_numerator    float    ,
inpatient_bed_covid_utilization_denominator    float    ,
geocoded_state    varchar    ,
latitude    float    ,
longitude    float    ,
state_desc varchar
);
"""

## Covid Data Dumping into Postgresql

In [4]:
dbConnection1 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432",database = "coviddb")
dbConnection1.set_isolation_level(0) # AUTOCOMMIT
dbCursor1 = dbConnection1.cursor()

try:
    #dbCursor1.execute('CREATE DATABASE coviddb;')
    dbCursor1.execute(createCovidTable)
    dbCursor1.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection1): dbConnection1.close()

In [5]:
try:
    dbConnection1 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432", database = "coviddb")
    print("Connected!")
    dbConnection1.set_isolation_level(0) # AUTOCOMMIT
    sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open("D:/AllDB/covid.csv", "r")
    with dbConnection1.cursor() as cur:
        cur.execute("truncate " + "coviddata" + ";")
        cur.copy_expert(sql=sql % "coviddata", file=file)
        dbConnection1.commit()       
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection1): dbConnection1.close()

Connected!


In [6]:
sql1 = """SELECT * FROM coviddata;"""
dbConnection1 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432",database = "coviddb")
covid_dataframe = sqlio.read_sql_query(sql1, dbConnection1)
covidTableStrData = pd.DataFrame().from_dict(covid_dataframe)
covidTableStrData

,dataasof,startdate,enddate,groups,year,month,hhsregion,state,placeofdeath,agegroup,covid19deaths,totaldeaths,pneumoniadeaths,pneumoniaandcovid19deaths,influenzadeaths,pneumoniainfluenzaorcovid19deaths
0,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,All Ages,526027.0,4035809.0,453484.0,255848.0,9004.0,731429.0
1,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,0-17 years,238.0,38250.0,646.0,44.0,179.0,1019.0
2,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,18-29 years,1916.0,72834.0,2109.0,850.0,150.0,3313.0
3,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,30-39 years,5583.0,103647.0,5088.0,2561.0,318.0,8406.0
4,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,40-49 years,15134.0,156430.0,12934.0,7445.0,494.0,21048.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33846,2021-03-24,2020-01-01,2020-01-31,By Month,2020,1,1,Vermont,Place of death unknown,50-64 years,0.0,0.0,0.0,0.0,0.0,0.0
33847,2021-03-24,2020-01-01,2020-01-31,By Month,2020,1,1,Vermont,Place of death unknown,65-74 years,0.0,0.0,0.0,0.0,0.0,0.0
33848,2021-03-24,2020-01-01,2020-01-31,By Month,2020,1,1,Vermont,Place of death unknown,75-84 years,0.0,0.0,0.0,0.0,0.0,0.0
33849,2021-03-24,2020-08-01,2020-08-31,By Month,2020,8,5,Wisconsin,Other,0-17 years,0.0,84.0,0.0,0.0,0.0,0.0


In [7]:
for column in covidTableStrData:
     print(column)

dataasof
startdate
enddate
groups
year
month
hhsregion
state
placeofdeath
agegroup
covid19deaths
totaldeaths
pneumoniadeaths
pneumoniaandcovid19deaths
influenzadeaths
pneumoniainfluenzaorcovid19deaths


## Mental State Data Dumping into Postgresql

In [8]:
dbConnection2 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432",database = "coviddb")
dbConnection2.set_isolation_level(0) # AUTOCOMMIT
dbCursor2 = dbConnection2.cursor()

try:
    dbCursor2.execute(createMentalTable)
    print("Table Created")
    dbCursor2.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection2): dbConnection2.close()

Table Created


In [9]:
try:
    dbConnection2 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432", database = "coviddb")
    print("Connected!")
    dbConnection2.set_isolation_level(0) # AUTOCOMMIT
    sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open("D:/AllDB/mentalState.csv", "r")
    with dbConnection2.cursor() as cur:
        cur.execute("truncate " + "mentalstatedata" + ";")
        cur.copy_expert(sql=sql % "mentalstatedata", file=file)
        dbConnection2.commit()       
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection2): dbConnection2.close()

Connected!


In [10]:
sql2 = """SELECT * FROM mentalstatedata;"""
dbConnection2 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432",database = "coviddb")
mentalstate_dataframe = sqlio.read_sql_query(sql2, dbConnection2)
mentalStateTableStrData = pd.DataFrame().from_dict(mentalstate_dataframe)
mentalStateTableStrData

,indicator,groups,state,subgroup,phase,timeperiod,timeperiodlabel,timeperiodstartdate,timeperiodenddate,value,lowci,highci,confidenceinterval,quartilerange
0,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,23.5,22.7,24.3,23.50,27.85
1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,32.7,30.2,35.2,32.70,27.85
2,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,25.7,24.1,27.3,25.70,27.85
3,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,24.8,23.3,26.2,24.75,27.85
4,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,23.2,21.5,25.0,23.25,27.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5569,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Virginia,Virginia,3 (Jan 6 Ã¯Â¿Â½ Mar 15),26.0,Mar 3 - Mar 15,2021-03-03,2021-03-15,34.3,30.5,38.2,34.35,34.70
5570,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Washington,Washington,3 (Jan 6 Ã¯Â¿Â½ Mar 15),26.0,Mar 3 - Mar 15,2021-03-03,2021-03-15,37.3,34.1,40.6,37.35,37.40
5571,Symptoms of Anxiety Disorder or Depressive Dis...,By State,West Virginia,West Virginia,3 (Jan 6 Ã¯Â¿Â½ Mar 15),26.0,Mar 3 - Mar 15,2021-03-03,2021-03-15,43.5,36.6,50.6,43.60,43.00
5572,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Wisconsin,Wisconsin,3 (Jan 6 Ã¯Â¿Â½ Mar 15),26.0,Mar 3 - Mar 15,2021-03-03,2021-03-15,31.7,27.0,36.6,31.80,29.60


In [11]:
for column in mentalStateTableStrData:
     print(column)

indicator
groups
state
subgroup
phase
timeperiod
timeperiodlabel
timeperiodstartdate
timeperiodenddate
value
lowci
highci
confidenceinterval
quartilerange


## Nursing Data Dumping into Postgresql

In [12]:
dbConnection3 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432",database = "coviddb")
dbConnection3.set_isolation_level(0) # AUTOCOMMIT
dbCursor3 = dbConnection3.cursor()

try:
    #dbCursor3.execute('CREATE DATABASE nursingdatadb;')
    dbCursor3.execute(createNursingTable)
    print("Table Created")
    dbCursor3.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection3): dbConnection3.close()

Table Created


In [13]:
try:
    dbConnection3 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432", database = "coviddb")
    print("Connected!")
    dbConnection3.set_isolation_level(0) # AUTOCOMMIT
    sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open("D:/AllDB/nursingData.csv", "r")
    with dbConnection3.cursor() as cur:
        cur.execute("truncate " + "nursingdata" + ";")
        cur.copy_expert(sql=sql % "nursingdata", file=file)
        dbConnection3.commit()       
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection3): dbConnection3.close()

Connected!


In [14]:
sql3 = """SELECT * FROM nursingdata;"""
dbConnection3 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432",database = "coviddb")
nursing_dataframe = sqlio.read_sql_query(sql3, dbConnection3)
nursingTableStrData = pd.DataFrame().from_dict(nursing_dataframe)
nursingTableStrData

,weekending,providername,providercity,residentstotaladmissionscovid19,residentsweeklyconfirmedcovid19,residentstotalconfirmedcovid19,residentstotalsuspectedcovid19,residentsweeklyalldeaths,residentstotalalldeaths,residentsweeklycovid19deaths,...,anycurrentsupplyofhandsanitizer,totalresidentconfirmedcovid19casesper1000residents,totalresidentcovid19deathsper1000residents,totalresidentscovid19deathsasapercentageofconfirmedcovid19cases,threeormoreconfirmedcovid19casesthisweekorinitialconfirmedcovid,abletotestorobtainresourcestotestallcurrentresidentswithinnext7,abletotestorobtainresourcestotestallstaffandorpersonnelwithinne,covid19pointofcaretestsperformedonresidentssincelastreport,covid19pointofcaretestsperformedonstaffandorpersonnelsincelastr,weeklyresidentconfirmedcovid19casesper1000residents
0,2020-05-24,NORTHBRIDGE HEALTH CARE CENTER,BRIDGEPORT,25,0,0,0,0,0,0,...,Y,389.8,89.1,25.0,N,Y,Y,1.0,3.0,0.0
1,2020-09-13,REGALCARE AT GREENWICH,GREENWICH,11,0,12,0,0,28,0,...,Y,193.5,225.8,116.7,N,Y,Y,0.0,0.0,0.0
2,2020-11-15,"WITHERELL, NATHANIEL",GREENWICH,161,0,30,14,0,16,0,...,Y,197.4,105.3,53.3,N,Y,Y,300.0,300.0,0.0
3,2020-12-27,AVERY NURSING HOME,HARTFORD,25,15,82,1,1,39,0,...,Y,581.6,148.9,25.6,Y,Y,Y,123.0,0.0,106.4
4,2020-08-30,REGALCARE AT GREENWICH,GREENWICH,11,0,12,0,0,28,0,...,Y,187.5,218.8,116.7,N,Y,Y,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256,2020-08-23,NOTRE DAME CONVALESCENT HOME I,NORWALK,3,0,22,38,0,17,0,...,Y,488.9,288.9,59.1,N,Y,Y,1.0,3.0,0.0
6257,2021-01-24,LEDGECREST HEALTH CARE,KENSINGTON,6,2,26,1,0,15,0,...,Y,541.7,104.2,19.2,N,Y,Y,0.0,0.0,41.7
6258,2021-01-31,JOHN L. LEVITOW HEALTH CARE CENTER,ROCKY HILL,12,1,34,0,0,20,0,...,Y,419.8,86.4,20.6,N,Y,Y,1.0,19.0,12.3
6259,2020-12-06,APPLE REHAB SHELTON LAKES,SHELTON,8,17,21,0,1,10,0,...,Y,265.8,0.0,0.0,Y,Y,Y,7.0,8.0,215.2


In [40]:
for column in nursingTableStrData:
     print(column)

weekending
providername
providercity
residentstotaladmissionscovid19
residentsweeklyconfirmedcovid19
residentstotalconfirmedcovid19
residentstotalsuspectedcovid19
residentsweeklyalldeaths
residentstotalalldeaths
residentsweeklycovid19deaths
residentstotalcovid19deaths
numberofallbeds
totalnumberofoccupiedbeds
staffweeklyconfirmedcovid19
stafftotalconfirmedcovid19
staffweeklysuspectedcovid19
stafftotalsuspectedcovid19
stafftotalcovid19deaths
shortageofnursingstaff
shortageofclinicalstaff
shortageofaides
shortageofotherstaff
anycurrentsupplyofn95masks
oneweeksupplyofn95masks
anycurrentsupplyofsurgicalmasks
oneweeksupplyofsurgicalmasks
anycurrentsupplyofeyeprotection
oneweeksupplyofeyeprotection
oneweeksupplyofgowns
anycurrentsupplyofhandsanitizer
totalresidentconfirmedcovid19casesper1000residents
totalresidentcovid19deathsper1000residents
totalresidentscovid19deathsasapercentageofconfirmedcovid19cases
threeormoreconfirmedcovid19casesthisweekorinitialconfirmedcovid
abletotestorobtainresou

## Covid Hospital Data Dumping into Postgresql

In [15]:
dbConnection4 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432",database = "coviddb")
dbConnection4.set_isolation_level(0) # AUTOCOMMIT
dbCursor4 = dbConnection4.cursor()

try:
    #dbCursor4.execute('CREATE DATABASE covidhospitaldb;')
    dbCursor4.execute(createCovidHospitalTable)
    print("Table Created")
    dbCursor4.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection4): dbConnection4.close()

Table Created


In [16]:
try:
    dbConnection4 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432", database = "coviddb")
    print("Connected!")
    dbConnection4.set_isolation_level(0) # AUTOCOMMIT
    sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open("D:/AllDB/covidHospital1.csv", "r")
    with dbConnection4.cursor() as cur:
        cur.execute("truncate " + "covidhospitaldata" + ";")
        cur.copy_expert(sql=sql % "covidhospitaldata", file=file)
        dbConnection4.commit()       
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection4): dbConnection4.close()

Connected!


In [17]:
sql4 = """SELECT * FROM covidhospitaldata;"""
dbConnection4 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432",database = "coviddb")
covidhospital_dataframe = sqlio.read_sql_query(sql4, dbConnection4)
covidHospitalTableStrData = pd.DataFrame().from_dict(covidhospital_dataframe)
covidHospitalTableStrData

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,percent_of_inpatients_with_covid_numerator,percent_of_inpatients_with_covid_denominator,inpatient_bed_covid_utilization,inpatient_bed_covid_utilization_coverage,inpatient_bed_covid_utilization_numerator,inpatient_bed_covid_utilization_denominator,geocoded_state,latitude,longitude,state_desc
0,IA,2021-01-03,6.0,63.0,57.0,6.0,62.0,58.0,2.0,124.0,...,594.0,4531.0,0.074028,124.0,594.0,8024.0,POINT (-93.500061 42.074659),-93.500061,42.074659,Iowa
1,ID,2021-01-03,4.0,47.0,1.0,7.0,44.0,1.0,5.0,51.0,...,356.0,1941.0,0.097695,51.0,356.0,3644.0,POINT (-114.659366 44.389073),-114.659366,44.389073,Idaho
2,IL,2021-01-03,19.0,171.0,17.0,17.0,173.0,17.0,73.0,202.0,...,4031.0,20611.0,0.125659,202.0,4031.0,32079.0,POINT (-89.148632 40.124144),-89.148632,40.124144,Illinois
3,IN,2021-01-03,22.0,142.0,2.0,29.0,135.0,2.0,9.0,164.0,...,2817.0,11444.0,0.152633,164.0,2817.0,18456.0,POINT (-86.2818 39.919991),-86.281800,39.919991,Indiana
4,KS,2021-01-03,10.0,136.0,4.0,13.0,133.0,4.0,14.0,147.0,...,1042.0,4940.0,0.119249,147.0,1042.0,8738.0,POINT (-98.38018 38.484729),-98.380180,38.484729,Kansas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17503,NJ,2020-09-21,11.0,81.0,1.0,11.0,81.0,1.0,41.0,92.0,...,627.0,15286.0,0.028026,91.0,627.0,22372.0,POINT (-74.655514 40.110253),-74.655514,40.110253,New Jersey
17504,NM,2020-09-21,13.0,36.0,2.0,13.0,36.0,2.0,1.0,49.0,...,122.0,2210.0,0.034620,50.0,122.0,3524.0,POINT (-106.108388 34.421369),-106.108388,34.421369,New Mexico
17505,NV,2020-09-21,4.0,52.0,4.0,5.0,51.0,4.0,40.0,58.0,...,460.0,5618.0,0.055900,58.0,460.0,8229.0,POINT (-114.916992 39.283294),-114.916992,39.283294,Nevada
17506,NY,2020-09-21,9.0,121.0,46.0,8.0,122.0,46.0,29.0,155.0,...,1272.0,30984.0,0.028809,156.0,1227.0,42591.0,POINT (-75.59655 42.921241),-75.596550,42.921241,New York


In [18]:
for column in covidHospitalTableStrData:
     print(column)

state
date
critical_staffing_shortage_today_yes
critical_staffing_shortage_today_no
critical_staffing_shortage_today_not_reported
critical_staffing_shortage_anticipated_within_week_yes
critical_staffing_shortage_anticipated_within_week_no
critical_staffing_shortage_anticipated_within_week_not_reported
hospital_onset_covid
hospital_onset_covid_coverage
inpatient_beds
inpatient_beds_coverage
inpatient_beds_used
inpatient_beds_used_coverage
inpatient_beds_used_covid
inpatient_beds_used_covid_coverage
previous_day_admission_adult_covid_confirmed_coverage
previous_day_admission_adult_covid_suspected_coverage
previous_day_admission_pediatric_covid_confirmed_coverage
previous_day_admission_pediatric_covid_suspected_coverage
staffed_adult_icu_bed_occupancy_coverage
staffed_icu_adult_patients_confirmed_and_suspected_covid_covera
staffed_icu_adult_patients_confirmed_covid_coverage
total_adult_patients_hospitalized_confirmed_and_suspected_covid
total_adult_patients_hospitalized_confirmed_covid_co

## Apply Joining and Lookup on Postgresql Database

In [19]:
display(covidTableStrData.head(), mentalStateTableStrData.head(), nursingTableStrData.head(), covidHospitalTableStrData.head())

,dataasof,startdate,enddate,groups,year,month,hhsregion,state,placeofdeath,agegroup,covid19deaths,totaldeaths,pneumoniadeaths,pneumoniaandcovid19deaths,influenzadeaths,pneumoniainfluenzaorcovid19deaths
0,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,All Ages,526027.0,4035809.0,453484.0,255848.0,9004.0,731429.0
1,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,0-17 years,238.0,38250.0,646.0,44.0,179.0,1019.0
2,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,18-29 years,1916.0,72834.0,2109.0,850.0,150.0,3313.0
3,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,30-39 years,5583.0,103647.0,5088.0,2561.0,318.0,8406.0
4,2021-03-24,2020-01-01,2021-03-20,By Total,2020,1,0,United States,Total - All Places of Death,40-49 years,15134.0,156430.0,12934.0,7445.0,494.0,21048.0


,indicator,groups,state,subgroup,phase,timeperiod,timeperiodlabel,timeperiodstartdate,timeperiodenddate,value,lowci,highci,confidenceinterval,quartilerange
0,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,23.5,22.7,24.3,23.50,27.85
1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,32.7,30.2,35.2,32.70,27.85
2,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,25.7,24.1,27.3,25.70,27.85
3,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,24.8,23.3,26.2,24.75,27.85
4,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,1.0,Apr 23 - May 5,2020-04-23,2020-05-05,23.2,21.5,25.0,23.25,27.85


,weekending,providername,providercity,residentstotaladmissionscovid19,residentsweeklyconfirmedcovid19,residentstotalconfirmedcovid19,residentstotalsuspectedcovid19,residentsweeklyalldeaths,residentstotalalldeaths,residentsweeklycovid19deaths,...,anycurrentsupplyofhandsanitizer,totalresidentconfirmedcovid19casesper1000residents,totalresidentcovid19deathsper1000residents,totalresidentscovid19deathsasapercentageofconfirmedcovid19cases,threeormoreconfirmedcovid19casesthisweekorinitialconfirmedcovid,abletotestorobtainresourcestotestallcurrentresidentswithinnext7,abletotestorobtainresourcestotestallstaffandorpersonnelwithinne,covid19pointofcaretestsperformedonresidentssincelastreport,covid19pointofcaretestsperformedonstaffandorpersonnelsincelastr,weeklyresidentconfirmedcovid19casesper1000residents
0,2020-05-24,NORTHBRIDGE HEALTH CARE CENTER,BRIDGEPORT,25,0,0,0,0,0,0,...,Y,389.8,89.1,25.0,N,Y,Y,1.0,3.0,0.0
1,2020-09-13,REGALCARE AT GREENWICH,GREENWICH,11,0,12,0,0,28,0,...,Y,193.5,225.8,116.7,N,Y,Y,0.0,0.0,0.0
2,2020-11-15,"WITHERELL, NATHANIEL",GREENWICH,161,0,30,14,0,16,0,...,Y,197.4,105.3,53.3,N,Y,Y,300.0,300.0,0.0
3,2020-12-27,AVERY NURSING HOME,HARTFORD,25,15,82,1,1,39,0,...,Y,581.6,148.9,25.6,Y,Y,Y,123.0,0.0,106.4
4,2020-08-30,REGALCARE AT GREENWICH,GREENWICH,11,0,12,0,0,28,0,...,Y,187.5,218.8,116.7,N,Y,Y,1.0,3.0,0.0


,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,percent_of_inpatients_with_covid_numerator,percent_of_inpatients_with_covid_denominator,inpatient_bed_covid_utilization,inpatient_bed_covid_utilization_coverage,inpatient_bed_covid_utilization_numerator,inpatient_bed_covid_utilization_denominator,geocoded_state,latitude,longitude,state_desc
0,IA,2021-01-03,6.0,63.0,57.0,6.0,62.0,58.0,2.0,124.0,...,594.0,4531.0,0.074028,124.0,594.0,8024.0,POINT (-93.500061 42.074659),-93.500061,42.074659,Iowa
1,ID,2021-01-03,4.0,47.0,1.0,7.0,44.0,1.0,5.0,51.0,...,356.0,1941.0,0.097695,51.0,356.0,3644.0,POINT (-114.659366 44.389073),-114.659366,44.389073,Idaho
2,IL,2021-01-03,19.0,171.0,17.0,17.0,173.0,17.0,73.0,202.0,...,4031.0,20611.0,0.125659,202.0,4031.0,32079.0,POINT (-89.148632 40.124144),-89.148632,40.124144,Illinois
3,IN,2021-01-03,22.0,142.0,2.0,29.0,135.0,2.0,9.0,164.0,...,2817.0,11444.0,0.152633,164.0,2817.0,18456.0,POINT (-86.2818 39.919991),-86.281800,39.919991,Indiana
4,KS,2021-01-03,10.0,136.0,4.0,13.0,133.0,4.0,14.0,147.0,...,1042.0,4940.0,0.119249,147.0,1042.0,8738.0,POINT (-98.38018 38.484729),-98.380180,38.484729,Kansas


In [4]:
sql5 = """ select * from
(
select cd.enddate, cd.state,  sum(cd.totaldeaths) as TotalDeaths, sum(cd.covid19deaths) as CovidDeaths, sum(cd.pneumoniadeaths) as PneumoniaDeaths, sum(cd.pneumoniaandcovid19deaths) as PneumoniaandCovidDeaths, sum(cd.influenzadeaths) as InfluenzaDeaths , sum(cd.pneumoniainfluenzaorcovid19deaths)as PICDeaths 
from coviddata cd
group by  cd.enddate,cd.state)a

join
(
select ms.state, ms.indicator, sum(ms.confidenceinterval) as ConfidenceInterval, sum(ms.quartilerange) as QuartileRange from mentalstatedata ms
group by ms.state, ms.indicator
) b on a.state = b.state 

join
(select state_desc, sum(critical_staffing_shortage_today_yes) as StaffShortage, sum(critical_staffing_shortage_today_no)as NoStaffShortage , sum(hospital_onset_covid_coverage) as hospital_onset_covid_coverage, sum(inpatient_beds_used_coverage)as inpatient_beds_used_coverage,  sum(inpatient_beds_used_covid) as inpatient_beds_used_covid
 from covidhospitaldata 
group by state_desc)d on d.state_desc = a.state

left join

(select weekending, sum(residentstotalalldeaths) as ResidentTotalDeath, sum(residentstotalcovid19deaths)as ResidentTotalCovidDeaths , sum(stafftotalconfirmedcovid19) as TotalConfirmedCovid19, sum(stafftotalsuspectedcovid19)as StaffSuspectedCovid,  sum(numberofallbeds) as Numberofallbeds, sum(totalnumberofoccupiedbeds) as TotalOccupiedBeds
 from nursingdata 
 group by weekending)c on a.enddate = c.weekending and a.state = 'Connecticut'; """
dbConnection5 = psycopg2.connect(user = "postgres",password = "abcd1234",host = "localhost",port = "5432",database = "coviddb")
covidDataFrame = sqlio.read_sql_query(sql5, dbConnection5)
covidTableStrData = pd.DataFrame().from_dict(covidDataFrame)
covidTableStrData

,enddate,state,totaldeaths,coviddeaths,pneumoniadeaths,pneumoniaandcoviddeaths,influenzadeaths,picdeaths,state,indicator,...,hospital_onset_covid_coverage,inpatient_beds_used_coverage,inpatient_beds_used_covid,weekending,residenttotaldeath,residenttotalcoviddeaths,totalconfirmedcovid19,staffsuspectedcovid,numberofallbeds,totaloccupiedbeds
0,2020-12-31,West Virginia,102202.0,6108.0,8769.0,2612.0,197.0,12525.0,West Virginia,Symptoms of Anxiety Disorder,...,17341.0,17820.0,110857.0,None,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-31,West Virginia,102202.0,6108.0,8769.0,2612.0,197.0,12525.0,West Virginia,Symptoms of Anxiety Disorder or Depressive Dis...,...,17341.0,17820.0,110857.0,None,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-12-31,West Virginia,102202.0,6108.0,8769.0,2612.0,197.0,12525.0,West Virginia,Symptoms of Depressive Disorder,...,17341.0,17820.0,110857.0,None,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-31,Iowa,13181.0,0.0,836.0,0.0,10.0,872.0,Iowa,Symptoms of Anxiety Disorder,...,31925.0,35779.0,139098.0,None,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-31,Iowa,13181.0,0.0,836.0,0.0,10.0,872.0,Iowa,Symptoms of Anxiety Disorder or Depressive Dis...,...,31925.0,35779.0,139098.0,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,2021-02-28,Arizona,25572.0,6108.0,4691.0,3748.0,0.0,7034.0,Arizona,Symptoms of Depressive Disorder,...,28700.0,30402.0,705351.0,None,NaN,NaN,NaN,NaN,NaN,NaN
935,2021-02-28,Arizona,25572.0,6108.0,4691.0,3748.0,0.0,7034.0,Arizona,Symptoms of Anxiety Disorder,...,28700.0,30402.0,705351.0,None,NaN,NaN,NaN,NaN,NaN,NaN
936,2020-04-30,District of Columbia,5511.0,982.0,1161.0,976.0,0.0,1172.0,District of Columbia,Symptoms of Depressive Disorder,...,3439.0,3736.0,85541.0,None,NaN,NaN,NaN,NaN,NaN,NaN
937,2020-04-30,District of Columbia,5511.0,982.0,1161.0,976.0,0.0,1172.0,District of Columbia,Symptoms of Anxiety Disorder or Depressive Dis...,...,3439.0,3736.0,85541.0,None,NaN,NaN,NaN,NaN,NaN,NaN
